<a href="https://colab.research.google.com/github/tomerni/PythonProjects/blob/main/HujiHackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
!pip install geopandas

     |████████████████████████████████| 1.0MB 6.9MB/s 
     |████████████████████████████████| 6.5MB 17.0MB/s 
     |████████████████████████████████| 15.3MB 242kB/s 
     |████████████████████████████████| 92kB 9.1MB/s 
  Found existing installation: click 8.0.0
    Uninstalling click-8.0.0:
      Successfully uninstalled click-8.0.0


In [ ]:
# load data
DATA_PATH = "./None"
data = np.loadtxt(DATA_PATH, delimiter=",")
# seperate X and y - X is the features and y is the result
X, y_3weeks, y_5weeks = data[:, :15], data[:, 15], data[:,16]
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y, tes)

In [ ]:
import geopandas as gpd
shapefile = 'ne_110m_admin_0_countries.shp'

gdf = gpd.read_file(shapefile)[['ADMIN', 'ADM0_A3', 'geometry']]
gdf.columns = ['country', 'country_code', 'geometry']
gdf.head()

,country,country_code,geometry
0,Fiji,FJI,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,United Republic of Tanzania,TZA,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,Western Sahara,SAH,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,Canada,CAN,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,United States of America,USA,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [ ]:
import pandas as pd


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# load data
DATA_PATH = "data_for_learning.csv"
COLS_TO_REMOVE = ["CountryCode", "prediction_cat", "prediction_cat_14", "prediction_cat_28"]
df = pd.read_csv('data_for_learning.csv')
y_3weeks = df["prediction_cat_14"]
y_5weeks = df["prediction_cat_28"]
y_cur = df["prediction_cat"]


for (columnName, columnData) in df.iteritems():
  df[columnName] = df[columnName].fillna(-1)

print(df.isnull().sum().sum())

X = df.drop(columns=COLS_TO_REMOVE)

# separate X and y - X is the features and y is the result
X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X, y_3weeks, test_size=0.2)
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(X, y_5weeks, test_size=0.2)
all_data_sets = [[X_train_3, X_test_3, y_train_3, y_test_3],
                 [X_train_5, X_test_5, y_train_5, y_test_5]]


# cnn model
cnn_model = tf.keras.models.Sequential()
cnn_model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
cnn_model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
cnn_model.add(tf.keras.layers.Dense(128, activation=tf.nn.softmax))
# TODO: maybe change the loss to MSE
cnn_model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# random forest model
clf = RandomForestClassifier(n_estimators=100)


"""
    TESTING
"""
list_of_models = [cnn_model, clf]
fitted_models = []
for model_index in range(len(list_of_models)):
    for data_set in all_data_sets:
        model = list_of_models[model_index]
        if model_index == 0:
            continue
            # model.fit(x=data_set[0], y=data_set[2], batch_size=60,
            #           verbose=1, epochs=200, validation_split=0.25)
            
            # predicted_y = model.predict(data_set[1])
            # # tf.math.confusion_matrix(data_set[3], predicted_y)
            # plot_confusion_matrix(model, data_set[1], data_set[3],
            #                       cmap=plt.cm.get_cmap("Blues"),
            #                       normalize=True)
            # plt.show()
        else:
            print(data_set[0].shape)
            model.fit(data_set[0], data_set[2])
            predicted_y = model.predict(data_set[1])
            # plot_confusion_matrix(model, data_set[1], data_set[3],
                                  # cmap=plt.cm.get_cmap("Blues"), normalize='true')
            # plt.show()
            fitted_models.append(model)






0
(86662, 13)
(86662, 13)


In [ ]:
### predict the last day of every country ####
import pandas as pd

COLS_TO_REMOVE = ["CountryCode", "prediction_cat", "prediction_cat_14", "prediction_cat_28"]

gb = df.groupby("CountryCode")
last_row_every_country = gb.tail(1).sort_values("CountryCode")
country_codes = pd.DataFrame(last_row_every_country["CountryCode"])
final_data = last_row_every_country.drop(columns=COLS_TO_REMOVE, axis=1)
model_14, model_28 = fitted_models[0], fitted_models[1]
prediction_14, prediction_28 = model_14.predict(final_data), model_28.predict(final_data)
country_codes["prediction_14"] = prediction_14
country_codes["prediction_28"] = prediction_28
country_codes["cur_prediction"] = y_cur


In [ ]:
country_codes[country_codes["prediction_28"] == 3].count()

CountryCode       92
prediction_14     92
prediction_28     92
cur_prediction    92
dtype: int64